In [27]:
import random
from datetime import datetime, timedelta
import networkx as nx
from pyvis.network import Network
from collections import deque

In [28]:
transactions = []
names = [
    '신소율', '홍수아', '홍지호', '신민지', '신도윤', '한민준', '안현우', '최하은', '권현준', '홍지은',
    '윤지원', '김태윤', '이주원', '박서준', '최예은', '강지호', '배수아', '정하늘', '오유진', '서민재',
    '장하준', '민서연', '김다은', '송지후', '이하린', '윤지우', '한지호', '최다은', '정민준', '서지우'
]

start_time = datetime(2024, 1, 1, 9, 0, 0)

for _ in range(30):
    sender = random.choice(names)
    receiver = random.choice(names)

    # 출금인과 송금인이 같으면 재선택
    while sender == receiver:
        receiver = random.choice(names)
    
    amount = random.randint(10000, 5000000)
    time_delta = timedelta(minutes=random.randint(0, 60*24*30))
    transaction_time = (start_time + time_delta).strftime('%m-%d %H:%M')

    transactions.append((sender, receiver, amount, transaction_time))

In [29]:
transactions

[('윤지우', '김다은', 4784746, '01-10 06:20'),
 ('오유진', '배수아', 124114, '01-11 01:20'),
 ('신민지', '신소율', 3572388, '01-28 07:07'),
 ('이주원', '홍수아', 1408012, '01-15 03:51'),
 ('배수아', '정민준', 698280, '01-08 02:10'),
 ('신도윤', '박서준', 1649041, '01-18 09:50'),
 ('최하은', '한지호', 4726438, '01-23 18:44'),
 ('최하은', '권현준', 4596967, '01-29 16:32'),
 ('신소율', '한지호', 638871, '01-07 18:53'),
 ('홍수아', '신도윤', 1520896, '01-29 14:52'),
 ('권현준', '민서연', 743196, '01-21 20:15'),
 ('신도윤', '윤지우', 559110, '01-28 23:05'),
 ('신소율', '정민준', 922636, '01-16 13:34'),
 ('강지호', '안현우', 2308162, '01-08 07:27'),
 ('신민지', '정하늘', 4903593, '01-18 05:48'),
 ('서지우', '홍수아', 883097, '01-09 12:48'),
 ('안현우', '이주원', 3728118, '01-04 00:19'),
 ('신소율', '권현준', 4117956, '01-07 04:56'),
 ('서민재', '윤지원', 3326623, '01-08 19:26'),
 ('홍지은', '오유진', 2823372, '01-12 16:08'),
 ('홍지은', '최하은', 2822218, '01-17 09:46'),
 ('최다은', '신민지', 1341119, '01-03 04:27'),
 ('최다은', '서지우', 3040211, '01-24 07:16'),
 ('안현우', '정민준', 604983, '01-12 05:20'),
 ('강지호', '박서준', 450366, 

In [30]:
# 전체 그래프 그리기
net = Network(height='800px', width='100%', directed=True)

for sender, receiver, amount, time in transactions:
    net.add_node(sender, label=sender)
    net.add_node(receiver, label=receiver)
    net.add_edge(sender, receiver, title=f"{amount}원\n{time}", value=amount)

net.save_graph('full_graph.html')

In [52]:
# 부분 그래프 그리기
G = nx.DiGraph()
for sender, receiver, amount, time in transactions:
    G.add_edge(sender, receiver, amount=amount, time=time)

# BFS (너비 우선 탐색)
def find_edges(G, start_node, max_depth):
    visited = set()
    visited.add(start_node) # 시작 노드 방문 추가
    queue = deque()
    queue.append((start_node, 0))
    result_edges = set()

    while queue:
        node, depth = queue.popleft()
        if depth >= max_depth:
            continue
        # 출금 (내가 돈을 보낸 사람)
        for neighbor in G.successors(node): 
            result_edges.add((node, neighbor))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
        # 입금 (나에게 돈을 보낸 사람)
        for neighbor in G.predecessors(node):
            result_edges.add((neighbor, node))
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth+1))
    return list(result_edges)
    
start_node = '최예은'
max_depth = 3
sub_edges = find_edges(G, start_node, max_depth)

# pyvis 시각화
net = Network(height='800px', width='100%', directed=True)
added_nodes = set()
for sender, receiver in sub_edges:
    for node in (sender, receiver):
        if node not in added_nodes:
            net.add_node(node, lable=node)
            added_nodes.add(node)
    edge_data = G.get_edge_data(sender, receiver)
    if edge_data: # edge_data가 있으면
        net.add_edge(sender, receiver, title=f"{edge_data['amount']}원\n{edge_data['time']}", value=edge_data['amount'])
    else:
        # edge_data가 없을 수도 있으므로 예외 처리
        net.add_edge(sender, receiver)

# 중심 노드 강조
net.get_node(start_node)['color'] = 'red'

# 저장
output_path = 'graph.html'
net.save_graph(output_path)
output_path

'graph.html'